In [ ]:
!pip install gdown -q
!gdown --id 1QR76tqp0qmPIXVchwue4cSz0_y3Xp_Bm --output dataset.zip
!unzip dataset.zip -d dataset

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8s.pt")

In [ ]:
pip install ultralytics

In [ ]:
import torch
from PIL import Image
from torchvision.transforms import ToTensor


img = Image.open('/kaggle/working/dataset/test_images/image_0014.png')
results = model(img)

results[0].show()

print(results[0].boxes[0].xyxy.tolist()[0]) 


In [ ]:
zebel1 = Image.open('/kaggle/working/dataset/zebel/1.png')
zebel2 = Image.open('/kaggle/working/dataset/zebel/2.png')
zebel3 = Image.open('/kaggle/working/dataset/zebel/3.png')

width1, height1 = zebel1.size
width2, height2 = zebel2.size
width3, height3 = zebel3.size

print(f"Image 1: Width = {width1}, Height = {height1}")
print(f"Image 2: Width = {width2}, Height = {height2}")
print(f"Image 3: Width = {width3}, Height = {height3}")


In [ ]:
import os

import shutil
dataset_dir = '/kaggle/working/dataset/zebel_dataset'

os.makedirs(f"{dataset_dir}/images", exist_ok=True)
os.makedirs(f"{dataset_dir}/labels", exist_ok=True)

shutil.copy('/kaggle/working/dataset/zebel/1.png', f"{dataset_dir}/images/1.png")
shutil.copy('/kaggle/working/dataset/zebel/2.png', f"{dataset_dir}/images/2.png")
shutil.copy('/kaggle/working/dataset/zebel/3.png', f"{dataset_dir}/images/3.png")

# Create corresponding label files for each image (you can adjust the bounding boxes as needed)
def create_label_file(image_name, width, height):
    label_file = f"{dataset_dir}/labels/{image_name.split('.')[0]}.txt"
    with open(label_file, 'w') as file:
        # Example annotation for "zebel_class" (class_id=0)
        # x_center, y_center, width, height are normalized by the image dimensions
        # Here we use a dummy bounding box, you should modify this as per your labels
        # Assuming a box at the center of the image
        x_center = 0.5 * width  # Normalized to [0, 1]
        y_center = 0.5 * height
        bbox_width = width  # Normalized width of the bounding box
        bbox_height = height  # Normalized height of the bounding box
        file.write(f"0 {x_center} {y_center} {bbox_width} {bbox_height}\n")

# Create labels for each image
create_label_file('1.png', width1, height1)
create_label_file('2.png', width2, height2)
create_label_file('3.png', width3, height3)

# Create data.yaml for YOLO
data_yaml = {
    'train': f'{dataset_dir}/images',
    'val': f'{dataset_dir}/images',
    'nc': 1,  # Number of classes (1 class in this case)
    'names': ['zebel_class'],  # List of class names
}

# Save the data.yaml file
import yaml
with open(f'{dataset_dir}/data.yaml', 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file)

print(f"Dataset directory structure created at: {dataset_dir}")


In [ ]:
from ultralytics import YOLO

# Define your custom class (e.g., "zebel_class")
class_names = ["zebel_class"]

# Initialize the model
model = YOLO("yolov8s.pt")

# Dataset path
dataset_path = '/kaggle/working/dataset/zebel_dataset'

# Create the dataset YAML file
# You need to create a YAML file that specifies the paths for training and validation images and labels
dataset_yaml = {
    'train': f'{dataset_path}/images',  # Path to the train images
    'val': f'{dataset_path}/images',  # Path to the train images
    'nc': 1,                            # Number of classes
    'names': class_names                # List of class names
}

# Save the YAML file
with open(f'{dataset_path}/data.yaml', 'w') as yaml_file:
    yaml.dump(dataset_yaml, yaml_file)

# Train the model with custom dataset
model.train(
    data=f'{dataset_path}/data.yaml',  # Path to the dataset YAML file
    epochs=10,                         # Number of training epochs
    batch=4,                           # Batch size
    imgsz=640,                         # Image size (you can experiment with this)
    classes=[0],                       # List of classes to train (index 0 for "zebel_class")
    name="zebel_class_model"            # Name of the model checkpoint
)

# After training, you can test the model again
model.eval()

# Testing with one image from the validation set
img = Image.open('/kaggle/working/dataset/test_images/image_0014.png')
results = model(img)

# Show the results
results[0].show()

# Print the bounding box of the first detection
print(results[0].boxes[0].xyxy.tolist()[0])


In [ ]:
import os

# Path to the directory containing test images
test_images_dir = '/kaggle/working/dataset/test_images'

# Get a list of image filenames in the directory
image_names = [f for f in os.listdir(test_images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Print the image names
print(image_names)


In [ ]:
# Assuming the model has been loaded and the image names are available
import math
import numpy as np
ll=[]
count = 0
for name in image_names:
    img = Image.open(test_images_dir + '/' + name)
    results = model(img)
    confmax1 = 0
    confmax2 = 0
    xx1 = yy1 = xx2 = yy2 = -1 
    degree = 0
    # Print the bounding box, class name, and confidence for each detection
    for i, box in enumerate(results[0].boxes):
        x1, y1, x2, y2 = box.xyxy.tolist()[0]
        class_id = box.cls.item()  # Get the class index for the bounding box
        class_name = results[0].names[class_id]  # Map class index to class name
        confidence = box.conf.item()  # Get the confidence score for the box

        if (class_name == 'person' or class_name == 'kite') and confidence >= confmax1:
            confmax1 = confidence
            xx1 = (x1 + x2)/2 
            yy1 = (y1 + y2)/2

        if class_name == 'airplane' and confidence >= confmax2:
            confmax2 = confidence
            xx2 = (x1 + x2)/2 
            yy2 = (y1 + y2)/2

    if xx1!=-1 and xx2!=-1:

        degree = math.atan((yy2 - yy1)/ np.abs(xx1-xx2))
    else:
        if xx2 != -1:
            degree = math.atan((yy2 - 150)/ np.abs(400-xx2))
        elif xx1 != -1:
            degree = math.atan((150 - yy1)/ np.abs(xx1-20))
        count+=1
        
    print(degree*180/np.pi)


    results[0].show()

    ll.append(-degree*180/np.pi)    
        # print(f"Image: {name}, Box {i+1}:")
        # print(f"Coordinates: ({x1}, {y1}), ({x2}, {y2})")
        # print(f"Class ID: {class_id}, Class Name: {class_name}")
        # print(f"Confidence: {confidence:.4f}")  # Print confidence with 4 decimal places
    

In [ ]:
import csv

# Prepare the CSV file
csv_file_path = "/kaggle/working/dataset/detected_labels.csv"
header = ["index", "file_name", "label"]

# Initialize the data list
data = []

# Assuming the model has been loaded and the image names are available
index = 0  # To keep track of row index in the CSV
    
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(image_names)  # Write the header row
    writer.writerows(ll)  # Write the data rows

print(f"CSV file saved at {csv_file_path}")


In [ ]:
count

In [ ]:
len(ll)

In [ ]:
import pandas as pd

# ll = list(map(int, ll))

df = pd.DataFrame({
    'file_name': image_names,
    'degree': ll
})

df = df.sort_values(by='file_name')

# df = df.reset_index(drop=True)

csv_file_path = '/kaggle/working/submission.csv'
df.to_csv(csv_file_path, index=False)